In [1]:
import pandas as pd
import pathlib

# Recuperação dos discursos

In [2]:
try:
    files = list(pathlib.Path("./discursos/").iterdir())
except FileNotFoundError:
    raise FileNotFoundError("The directory 'discursos' was not found. It's necessary for the correct function of this file.")

In [3]:
df_list = [pd.read_csv(file) for file in files]

In [4]:
partidos = []
for df in df_list:
    partidos.extend(df["sigla"].unique().tolist())

In [6]:
discursos = [df["transcricao"].to_list() for df in df_list]

# Pre-processing

Sabe-se que O começo do discurso é feito com:

O SR(A).NOME SOBRENOME(SIGLA-UF. Sem revisão do orador) - Começo do discurso

Sabe-se portanto retirar este começo, utilizando-se do fato de haver

In [8]:
from preprocess import preprocess

In [9]:
discursos = [preprocess(discursos_) for discursos_ in discursos]

# Lemmatization

In [10]:
import spacy

In [11]:
def lemmatization(texts, nlp, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    texts_out = []
    for sent in texts:
        doc = nlp(sent)
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ["-PRON-"] else "" for token in doc if token.pos_ in allowed_postags]))
    return texts_out
# Ideia possível, o token possui um campo head, se a head da palavra for Presidente e a palavra for Jair e/ou Bolsonaro, deve-se manter a palavra no texto e não descartá-la como está acontecendo com outras palavras

In [12]:
nlp = spacy.load("pt_core_news_lg")

In [13]:
discursos = [lemmatization(discursos_, nlp, allowed_postags=["NOUN", "ADJ", "ADV", "VERB", "PUNCT"]) for discursos_ in discursos]

# Remoção das stop_words

In [14]:
from nltk.corpus import stopwords

In [15]:
stop_words = set(stopwords.words("portuguese"))

In [16]:
def remove_stop_words(discursos: list) -> list:
    novos_discursos = []
    for discurso in discursos:
        discurso_split = discurso.split()
        novo_discurso = []
        for palavra in discurso_split:
            if palavra not in stop_words:
                novo_discurso.append(palavra)
        novo_discurso = " ".join(novo_discurso)
        novos_discursos.append(novo_discurso)
    return novos_discursos

In [17]:
discursos = [remove_stop_words(discursos_) for discursos_ in discursos]

# Tokenização

In [19]:
import nltk

In [20]:
def tokenizer(discursos: list):
    discursos_lower = [discurso.lower() for discurso in discursos]
    discursos_tokenized = [nltk.word_tokenize(discurso) for discurso in discursos_lower]
    return discursos_tokenized

In [21]:
discursos = [tokenizer(discursos_) for discursos_ in discursos]

# Remoção das pontuações

In [22]:
import string

In [23]:
def remove_punct(discursos: list) -> list:
    novos_discursos =[]
    for discurso in discursos:
        novo_discurso = []
        for token in discurso:
            if token not in string.punctuation:
                novo_discurso.append(token)
        novos_discursos.append(novo_discurso)
    return novos_discursos

In [24]:
discursos = [remove_punct(discursos_) for discursos_ in discursos]

In [25]:
def unifica_discursos(discursos: list) -> list:
    novos_discursos =[" ".join(discurso) for discurso in discursos]
    return novos_discursos

In [26]:
discursos = [unifica_discursos(discursos_) for discursos_ in discursos]

## Correção de alguns bugs gerados pela Lemmatização do Spacy

In [28]:
def correcoes_lemmatizacao(discursos: list) -> list:
    novos_discursos = [discurso.replace("morer", "moro") for discurso in discursos]
    return novos_discursos

In [29]:
discursos = [correcoes_lemmatizacao(discursos_) for discursos_ in discursos]

# Vetorização

In [30]:
import sklearn.feature_extraction.text as sklearntext

In [31]:
vectorizer = sklearntext.CountVectorizer(analyzer='word', stop_words=None, lowercase=True)

In [32]:
def data_vectorizer(vectorizer: sklearntext.CountVectorizer, discursos: list):
    matrix_discursos = vectorizer.fit_transform(discursos)
    feature_names = vectorizer.get_feature_names_out()
    return matrix_discursos, feature_names

In [33]:
# Talvez usar dict comprehesion
data_vectorized = []
feature_names = []
for discursos_ in discursos:
    data, feature_name = data_vectorizer(vectorizer, discursos_)
    data_vectorized.append(data)
    feature_names.append(feature_name)

# Extração de tópicos

In [34]:
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

In [35]:
def lda_transform_fit(data_vectorized: np.ndarray) -> LatentDirichletAllocation:
    lda_model = LatentDirichletAllocation(learning_method="online", random_state=100, batch_size=128, evaluate_every = -1, n_jobs= -1, n_components=10)
    lda_output = lda_model.fit_transform(data_vectorized)
    return lda_model

In [36]:
def transform_topics(feature_names: np.ndarray, lda_model_components: np.ndarray, n_words = 20) -> list:
    keywords = np.array(feature_names)
    topic_keywords = []
    for topic_weights in lda_model_components:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [37]:
lda_models =[lda_transform_fit(data_vectorized_) for data_vectorized_ in data_vectorized]

In [38]:
topics_keyowrds_lst = [transform_topics(feature_name, lda_models[i].components_, 15) for i, feature_name in enumerate(feature_names)]

In [39]:
topics_keywords_df = [pd.DataFrame(topics_keywords) for topics_keywords in topics_keyowrds_lst]

In [ ]:
pathlib.Path("./topics/lda/").mkdir(parents=True, exist_ok=True)

In [40]:
for i, df in enumerate(topics_keywords_df):
    topics_path = pathlib.Path(f"./topics/lda/topics_{partidos[i]}.csv")
    df.columns = ["Word" + str(i) for i in range(df.shape[1])]
    df.index = ["Topic " + str(i) for i in range(df.shape[0])]
    df.to_csv(topics_path)